# Prompt Engineering Lab: Customer Feedback Analysis

In this lab, you'll learn how to iteratively improve prompts for analyzing customer feedback using a variety of prompt engineering techniques. You'll see how each technique can help you get clearer, more actionable responses from a language model.

**Customer Feedback Example:**
> I ordered a coffee maker last week, but it still hasn’t arrived. The tracking link doesn’t work, and I can’t get through to customer service. I’m really frustrated.

## Setup


This notebook is compatible with both [MyBinder](https://mybinder.org/) and AWS SageMaker.

**Environment Notes:**
- **MyBinder:** You must provide your own AWS credentials (see below).
- **AWS SageMaker:** Credentials are pre-configured in your environment.

**AWS Credentials Setup:**
- **MyBinder:** Set your credentials as environment variables in a cell (do NOT share credentials):
```python
import os
os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_ACCESS_KEY'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_SECRET_KEY'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'  # or your region
```
- **SageMaker:** No action needed unless you want to override the default region.

In [1]:
import os
import boto3

os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_ACCESS_KEY'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_SECRET_KEY'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

# Set the Claude model ID to use for all Bedrock calls
# Update this value as needed for your Bedrock account.
model_id = 'anthropic.claude-3-5-sonnet-20241022-v2:0'  # Example: Claude 3.5 Sonnet


region = os.environ.get('AWS_DEFAULT_REGION', 'us-west-2')
bedrock = boto3.client('bedrock-runtime', region_name=region)
# You can find available model IDs using the earlier Bedrock listing cell.

customer_feedback = "I ordered a coffee maker last week, but it still hasn’t arrived. \
The tracking link doesn’t work, and I can’t get through to customer service. I’m really frustrated."

In [2]:
import json

def call_bedrock_model(prompt):
    # Create the request body
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0,
        "top_k": 1
    }

    response = bedrock.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body),
        contentType='application/json'
    )
    result = json.loads(response['body'].read())
    model_response = result['content'][0]['text'].strip()
    print('Model response:')
    print(model_response)
    return model_response

## 1. The "Bad" Prompt (Prompt Design Challenges)
This prompt is vague and lacks structure. Run it and observe the model's output. What problems do you notice?

In [3]:
bad_prompt = (
    f"{customer_feedback}\n\n"
    "What’s wrong here? Can you help?")
# Call your model here and print the response
response = call_bedrock_model(bad_prompt)

Model response:
I understand your frustration with the delayed delivery and communication issues. While I can't directly track your order or contact customer service for you, I can suggest some steps to help:

1. Double-check your order confirmation email for the correct tracking number
2. Try tracking through the carrier's website directly (USPS, FedEx, UPS, etc.)
3. Contact the seller through alternative means:
- Email
- Social media channels
- Their website's chat function
- Alternative customer service numbers

Would you like me to help you think through which of these options might work best in your situation?


## 2. CRISP Framework Practice
Now, let's structure the prompt using the CRISP framework (without role assignment).

**Instructions:**
- Read the customer feedback below.
- Summarize the main problem and suggest one practical next step the company should take to help this customer.
- Format your answer as:
    - **Summary:**
    - **Main Problem:**
    - **Suggested Next Step:**

In [4]:
crisp_prompt = (
    f"Read the customer feedback below:\n"
    f"{customer_feedback}\n\n"
    f"Summarize the main problem and suggest practical next steps the company should take to help this customer and the escalation path to fix this issue permanently.\n"
    f"Please format your answer as:\n"
    f"Summary:<overview>\n"
    f"<bullet points>\n\n"
    f"Main Problem:<overview>\n"
    f"<bullet points>\n\n"
    f"Suggested Next Steps:<overview>\n"
    f"<bullet points>\n\n"
    "For each section, provide a one sentence <overview> and details as max 3 to 4 <bullet points> ordered by priority"
)
#Call your model here and print the response
response = call_bedrock_model(crisp_prompt)

Model response:
Summary: Customer experiencing multiple service failures related to a delayed coffee maker order.
• Order delivery is delayed beyond expected timeframe
• Tracking functionality is not working
• Customer service is unresponsive
• Customer expressing clear dissatisfaction

Main Problem: Multiple breakdowns in the order fulfillment and customer communication system requiring both immediate and systemic fixes.
• Primary issue is lack of order visibility and status
• Failed customer service touchpoints preventing resolution
• Broken tracking system indicating technical issues
• Multiple failure points suggesting systemic problems

Suggested Next Steps: Immediate customer resolution needed while addressing underlying system issues.
Immediate Actions:
• Directly contact customer with order status and estimated delivery date
• Provide working tracking information or alternative status updates
• Offer compensation (expedited shipping, discount, or free item)

Long-term Fixes:
• 

## 3. Role Assignment
Now, let's add a role to the prompt.

**Instructions:**
- Add a specific role to the prompt (e.g., customer support specialist).
- Observe how the model's response changes.

In [5]:
role_prompt = (
    f"You are an expert customer support specialist for an online retailer with 20 years of experience.\n"
    f"Read the customer feedback below:\n"
    f"{customer_feedback}\n\n"
    f"Summarize the main problem and suggest practical next steps the company should take to help this customer and the escalation path to fix this issue permanently.\n"
    f"Please format your answer as:\n"
    f"Summary:<overview>\n"
    f"<bullet points>\n\n"
    f"Main Problem:<overview>\n"
    f"<bullet points>\n\n"
    f"Suggested Next Step:<overview>\n"
    f"<bullet points>\n\n"
    f"For each section, provide a one sentence <overview> and details as max 3 to 4 <bullet points> ordered by priority"
)
#Call your model here and print the response
response = call_bedrock_model(role_prompt)

Model response:
Summary: Customer experiencing multiple service failures related to a delayed coffee maker order.
• Order delivery is delayed beyond expected timeframe
• Tracking functionality is not working
• Customer service is inaccessible
• Multiple touchpoints failing simultaneously creating compound frustration

Main Problem: Core issue is lack of order visibility combined with failed customer service access.
• Non-functional tracking system prevents customer self-service
• Customer service accessibility failure removes backup support option
• Customer has no way to determine order status or resolution timeline
• Communication breakdown on multiple channels

Suggested Next Steps: Immediate customer outreach needed with clear resolution path and system fixes.
• Contact customer directly with order status and specific delivery timeline
• Provide direct contact information for dedicated support representative
• Investigate and fix tracking system technical issues
• Review and enhanc

## 4. Self-Evaluation
Now, let's have the model critique its own answer or ask clarifying questions.

**Instructions:**
- After providing your answer, review it for any uncertainties or missing information.
- If anything is unclear, list clarifying questions you would ask the customer.

In [6]:
self_eval_prompt = (
    f"You are an expert customer support specialist for an online retailer with 20 years of experience.\n"
    f"Read the customer feedback below:\n"
    f"{customer_feedback}\n\n"
    f"Summarize the main problem and suggest practical next steps the company should take to help this customer and the escalation path to fix this issue permanently.\n"
    f"Please format your answer as:\n"
    f"Summary:<overview>\n"
    f"<bullet points>\n\n"
    f"Main Problem:<overview>\n"
    f"<bullet points>\n\n"
    f"Suggested Next Step:<overview>\n"
    f"<bullet points>\n\n"
    f"For each section, provide a one sentence <overview> and details as max 3 to 4 <bullet points> ordered by priority\n"
    "After your answer, review it for any uncertainties or missing information. If anything is unclear, list clarifying questions you would ask the customer."
)
#Call your model here and print the response
response = call_bedrock_model(self_eval_prompt)

Model response:
Summary: Customer experiencing multiple service failures related to a delayed coffee maker order.
• Order delivery is delayed beyond expected timeframe
• Tracking functionality is not working
• Customer service is inaccessible
• Customer expressing clear frustration with overall experience

Main Problem: Multiple system breakdowns creating compounded negative customer experience.
• Primary shipping/logistics failure (delayed delivery)
• Technical failure (broken tracking link)
• Service failure (unreachable customer support)
• Lack of proactive communication about issues

Suggested Next Steps: Immediate customer contact and resolution while addressing systemic issues.
• Contact customer directly with sincere apology and order status update
• Provide working tracking information or specific delivery timeline
• Offer compensation (expedited shipping, partial refund, or gift card)
• Escalate to logistics and IT teams to investigate tracking system failure

Clarifying Quest

## 5. Prompt Chaining
Break the task into sequential steps, using the output of one prompt as input for the next.

**Step 1:** Extract all issues mentioned in the customer feedback.

In [7]:
import json

chaining_step1_prompt = (
    f"Extract all issues mentioned in the following customer feedback and output should be just a JSON list of strings:\n"
    f"{customer_feedback}\n\n"
)
#Call your model here and print the response
response = call_bedrock_model(chaining_step1_prompt)
issues = json.loads(response)

Model response:
[
  "delayed delivery",
  "broken tracking link",
  "inaccessible customer service"
]


**Step 2:** For each issue identified, suggest a specific action the company should take.

In [12]:
import time
# Use the output from Step 1 as input here
for issue in issues:
    print(f"\n\nIssue: {issue}")
    prompt = (
        f"You are an expert customer support specialist for an online retailer with 20 years of experience.\n"
        f"Suggest top 2 questions to help the company get to the root cause of this issue: \"{issue}\""
    )
    response = call_bedrock_model(prompt)
    time.sleep(5)



Issue: delayed delivery
Model response:
As an experienced customer support specialist, I recommend these 2 key diagnostic questions to identify the root cause of delayed deliveries:

1. "At which point in the shipping journey did the delay occur? (Order processing, warehouse pickup, in transit, or last-mile delivery)"
- This helps pinpoint exactly where the breakdown happened in the fulfillment chain
- Reveals if it's an internal process issue or carrier-related problem
- Allows for targeted solutions based on the specific stage

2. "Is this delay pattern occurring with specific product categories, shipping carriers, or geographical regions?"
- Identifies if the issue is systemic or isolated
- Helps detect patterns (e.g., certain carriers consistently underperforming)
- Enables data-driven decisions for long-term fixes

These questions provide actionable insights while being specific enough to drive meaningful improvements in delivery performance.


Issue: broken tracking link
Model 

## 6. Chain-of-Thought Prompting
Guide the model to reason step-by-step.

In [9]:
cot_prompt = (
    "Read the customer feedback below:\n"
    f"{customer_feedback}\n\n"
    "Think step-by-step:\n"
    "1. Restate the customer’s main concern in your own words.\n"
    "2. Identify all the issues the customer is experiencing.\n"
    "4. For each issue, suggest top 2 questions to help the company get to the root cause of this issue"
)
#Call your model here and print the response
call_bedrock_model(cot_prompt)

Model response:
Let me break this down step by step:

1. Restating the main concern:
The customer is unable to receive or track their coffee maker order and cannot get assistance from customer service.

2. Identifying all issues:
- Delayed delivery of coffee maker
- Non-functioning tracking link
- Inability to reach customer service

3. Questions for each issue:

Delayed delivery:
- When exactly was the order placed (date and time)?
- Was there a specific delivery timeframe promised during checkout?

Non-functioning tracking link:
- What error message appears when trying to access the tracking link?
- Through which method was the tracking link provided (email, text, website)?

Customer service accessibility:
- What methods have you tried to contact customer service (phone, email, chat)?
- What times/days have you attempted to reach customer service?

These questions would help the company:
- Verify if the delivery is actually delayed against promised timeline
- Identify if there's a te

"Let me break this down step by step:\n\n1. Restating the main concern:\nThe customer is unable to receive or track their coffee maker order and cannot get assistance from customer service.\n\n2. Identifying all issues:\n- Delayed delivery of coffee maker\n- Non-functioning tracking link\n- Inability to reach customer service\n\n3. Questions for each issue:\n\nDelayed delivery:\n- When exactly was the order placed (date and time)?\n- Was there a specific delivery timeframe promised during checkout?\n\nNon-functioning tracking link:\n- What error message appears when trying to access the tracking link?\n- Through which method was the tracking link provided (email, text, website)?\n\nCustomer service accessibility:\n- What methods have you tried to contact customer service (phone, email, chat)?\n- What times/days have you attempted to reach customer service?\n\nThese questions would help the company:\n- Verify if the delivery is actually delayed against promised timeline\n- Identify if t

## 7. Multi-Path Reasoning (Tree of Thoughts)
Ask the model to consider multiple solution paths and select the best.

**Instructions:**
- List three different ways the company could address the customer’s main problem.
- For each approach, explain the pros and cons.
- Then, select the best approach and justify your choice.

In [10]:
tree_of_thoughts_prompt = (
    "Read the customer feedback below:\n"
    f"{customer_feedback}\n\n"
    "List three different ways the company could address the customer’s main problem.\n"
    "For each approach, explain the pros and cons.\n"
    "Then, select the best approach and justify your choice."
)
#Call your model here and print the response
call_bedrock_model(tree_of_thoughts_prompt)

Model response:
Here are three approaches to address the customer's delivery and communication issues:

1. Immediate Replacement Shipment
Pros:
- Quick resolution for the customer
- Shows commitment to customer satisfaction
- Eliminates further waiting time

Cons:
- Additional cost for the company
- Original package might still arrive (duplicate shipment)
- Doesn't address underlying tracking/communication issues

2. Priority Customer Service Response
Pros:
- Provides personal attention to the issue
- Can investigate and explain the specific problem
- Maintains communication with customer

Cons:
- May take time to research the issue
- Customer remains without their product
- Requires additional customer service resources

3. Automated Status Update + Compensation
Pros:
- Immediate acknowledgment of the problem
- Customer gets clear timeline and compensation
- Scalable solution for similar issues

Cons:
- Less personal approach
- May not satisfy customers wanting immediate resolution
- 

"Here are three approaches to address the customer's delivery and communication issues:\n\n1. Immediate Replacement Shipment\nPros:\n- Quick resolution for the customer\n- Shows commitment to customer satisfaction\n- Eliminates further waiting time\n\nCons:\n- Additional cost for the company\n- Original package might still arrive (duplicate shipment)\n- Doesn't address underlying tracking/communication issues\n\n2. Priority Customer Service Response\nPros:\n- Provides personal attention to the issue\n- Can investigate and explain the specific problem\n- Maintains communication with customer\n\nCons:\n- May take time to research the issue\n- Customer remains without their product\n- Requires additional customer service resources\n\n3. Automated Status Update + Compensation\nPros:\n- Immediate acknowledgment of the problem\n- Customer gets clear timeline and compensation\n- Scalable solution for similar issues\n\nCons:\n- Less personal approach\n- May not satisfy customers wanting immedi

## Student Reflection
Which technique produced the most useful or reliable answer for this scenario? How did the model’s responses change as you refined your prompts?